In [ ]:
import influxdb_client
import os

In [ ]:
bucket = os.getenv("bucket")
org = os.getenv("org")
token = os.getenv("token")
url = "http://localhost:8086"

In [ ]:
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

In [ ]:
target_measurement = "medaka"
target_field = "temperature"

In [ ]:
query = f"""
from(bucket: "{bucket}")
|> range(start: -1d)
|> filter(fn: (r) => r._measurement == "{target_measurement}")
|> filter(fn: (r) => r._field == "{target_field}")
"""

In [ ]:
result = query_api.query(org=org, query=query)

In [ ]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_field(), record.get_value(), record.get_time()))

print(results)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date, timedelta, time
import pytz
from typing import List

In [ ]:
yesterday = date.today() - timedelta(days=1)
begin_datetime_str = (pytz.timezone('Asia/Tokyo').localize(datetime.combine(yesterday, time()))).isoformat()
stop_datetime_str = (pytz.timezone('Asia/Tokyo').localize(datetime.combine(date.today(), time()))).isoformat()

In [ ]:
print(f"range: {begin_datetime_str} to {stop_datetime_str}")

In [ ]:
query = f"""
from(bucket: "{bucket}")
|> range(start: {yesterday_begin_str_utc}, stop: {today_begin_str_utc})
|> filter(fn: (r) => r._measurement == "{target_measurement}")
|> filter(fn: (r) => r._field == "{target_field}")
|> movingAverage(n: 5)
"""

In [ ]:
result = query_api.query(org=org, query=query)

In [ ]:
times = []
values = []
for table in result:
    for record in table.records:
        times.append(record.get_time())
        values.append(record.get_value())

In [ ]:
plt.plot(times, values)

In [ ]:
def get_data(bucket: str, target_measurement: str, target_field: str, days: int=7) -> List:
    times_series = []
    values_series = []
    for idx in range(1, days+1):
        begin_index_date = date.today() - timedelta(days=idx)
        stop_index_date = date.today() - timedelta(days=(idx-1))
        begin_datetime_str = (pytz.timezone('Asia/Tokyo').localize(datetime.combine(begin_index_date, time()))).isoformat()
        stop_datetime_str = (pytz.timezone('Asia/Tokyo').localize(datetime.combine(stop_index_date, time()))).isoformat()
        query = f"""
    from(bucket: "{bucket}")
    |> range(start: {begin_datetime_str}, stop: {stop_datetime_str})
    |> filter(fn: (r) => r._measurement == "{target_measurement}")
    |> filter(fn: (r) => r._field == "{target_field}")
    |> movingAverage(n: 5)
    """
        print(f"range: {begin_datetime_str} to {stop_datetime_str}")
        result = query_api.query(org=org, query=query)
        times = []
        values = []
        for table in result:
            for record in table.records:
                times.append(record.get_time())
                values.append(record.get_value())
        times_series.append(times)
        values_series.append(values)
    return times_series, values_series

In [ ]:
temp_times_series, temp_values_series = get_data("sensor-data", "medaka", "temperature")

In [ ]:
humid_times_series, humid_values_series = get_data("sensor-data", "medaka", "humidity")

In [ ]:
for idx in range(7):
    plt.plot(list(range(len(temp_values_series[idx]))), temp_values_series[idx])
    plt.plot(list(range(len(humid_values_series[idx]))), humid_values_series[idx])